In [ ]:
import sys
#sys.path.append('/home/mcgreivy/InvariantPreservingMLSolvers/2d_incompressible_euler/ml')
#sys.path.append('/home/mcgreivy/InvariantPreservingMLSolvers/2d_incompressible_euler/baselines')
#sys.path.append('/home/mcgreivy/InvariantPreservingMLSolvers/2d_incompressible_euler/simulate')
sys.path.append('/Users/nickm/thesis/InvariantPreservingMLSolvers/2d_incompressible_euler/ml')
sys.path.append('/Users/nickm/thesis/InvariantPreservingMLSolvers/2d_incompressible_euler/baselines')
sys.path.append('/Users/nickm/thesis/InvariantPreservingMLSolvers/2d_incompressible_euler/simulate')

import jax
import jax.numpy as jnp
import numpy as onp
from jax import config
config.update("jax_enable_x64", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

from initialconditions import init_fn_FNO, init_fn_jax_cfd
from simulations import KolmogorovFiniteVolumeSimulation
from simparams import FiniteVolumeSimulationParams

from helper import convert_FV_representation
from trajectory import get_trajectory_fn
from flux import Flux

from model import LearnedFlux2D
from mlparams import ModelParams, TrainingParams
from trainingutils import init_params, save_training_data, save_training_params
from trainingutils import get_loss_fn, get_batch_fn, get_idx_gen, train_model, compute_losses_no_model


#########################
# HYPERPARAMS
#########################


simname = "burnin_test"
train_id = "burnin_test"

cfl_safety=0.3
Lx = Ly = 2 * jnp.pi
viscosity=1/1000
forcing_coeff = 1.0
drag = 0.1
max_velocity = 7.0
ic_wavenumber = 2

batch_size= 100
learning_rate=1e-4
num_epochs = 100
kernel_size = 5
depth = 5
width = 64

nx_exact = ny_exact = 128
outer_steps = 100
n_runs = 1
t_inner = 0.01
t_burnin = 40.0
nxs = [32, 64]

#########################
# END HYPERPARAMS
#########################

In [ ]:
def get_sim_params(nx, ny, global_stabilization=False):
    basedir = "/home/mcgreivy/InvariantPreservingMLSolvers/2d_incompressible_euler"
    readwritedir = "/scratch/gpfs/mcgreivy/InvariantPreservingMLSolvers/2d_incompressible_euler"
    rk='ssp_rk3'
    flux=Flux.VANLEER
    return FiniteVolumeSimulationParams(simname, basedir, readwritedir, nx, ny, Lx, Ly, cfl_safety, rk, flux, global_stabilization)

def get_simulation(sim_params, model=None, params=None):
    return KolmogorovFiniteVolumeSimulation(sim_params, viscosity, forcing_coeff, drag, model=model, params=params)

def get_trajectory(sim_params, v0):
    v_init = convert_FV_representation(v0, sim_params)
    sim = get_simulation(sim_params)
    rollout_fn = get_trajectory_fn(sim.step_fn, outer_steps)
    return rollout_fn(v_init)

def get_ml_params():
    return ModelParams(train_id, batch_size, learning_rate, num_epochs, kernel_size, depth, width)

def get_model():
    model_params = get_ml_params()
    return LearnedFlux2D(model_params)

## Compare dadt with training data, new data

In [ ]:
t_inner = 0.01
outer_steps = 100
t_burn_in = 40.0

def errors_sim(key):
    sim_params_exact = get_sim_params(nx_exact, ny_exact)
    simulation_exact = get_simulation(sim_params_exact)

    key, subkey = jax.random.split(key)

    vorticity0 = init_fn_jax_cfd(subkey, sim_params_exact, 7.0, 2)

    inner_fn_burnin = get_inner_fn(simulation_exact.step_fn, simulation_exact.dt_fn, t_burn_in)
    rollout_burnin_fn = jax.jit(get_trajectory_fn(inner_fn_burnin, 1, start_with_input = False))
    v_burnin = rollout_burnin_fn(vorticity0)[0]

    inner_fn = get_inner_fn(simulation_exact.step_fn, simulation_exact.dt_fn, t_inner)
    rollout_fn = jax.jit(get_trajectory_fn(inner_fn, outer_steps))
    exact_trajectory = rollout_fn(v_burnin)

    convert_fn = lambda v: convert_FV_representation(v, sim_params_vanleer)
    model = get_model()

    ######


    errors_vanleer = onp.zeros((len(nxs), exact_trajectory.shape[0]))
    errors_model = onp.zeros((len(nxs), exact_trajectory.shape[0]))


    for t, a in enumerate(exact_trajectory):

        for i, nx in enumerate(nxs):
            _, params = load_training_params(sim_params, model)

            dadt_exact = jax.jit(simulation_exact.F)(a)
            dadt_exact_ds = convert_fn(dadt_exact)

            a_ds = convert_fn(a)
            dadt_model = jax.jit(lambda a: simulation_vanleer.F_params(a, model, params))(a_ds)
            dadt_vanleer = jax.jit(simulation_vanleer.F)(a_ds)

            errors_vanleer[i, t] = jnp.mean((dadt_vanleer - dadt_exact_ds)**2)
            errors_model[i, t] = jnp.mean((dadt_model - dadt_exact_ds)**2)

    return errors_vanleer, errors_model
     
    
key_train = jax.random.PRNGKey(0)
key_eval = jax.random.PRNGKey(105)

errors_vanleer_train, errors_model_train = errors_sim(key_train)
errors_vanleer_eval, errors_model_eval = errors_sim(key_eval)
    
for i, nx in enumerate(nxs):
    plt.plot(errors_vanleer_train, label="Training Data, nx={}, MUSCL".format(nx))
    plt.plot(errors_model_train, label="Training Data, nx={}, ML".format(nx))
    plt.plot(errors_vanleer_eval, label="Evalulation Data, nx={}, MUSCL".format(nx))
    plt.plot(errors_model_eval, label="Evaluation Data, nx={}, ML".format(nx))
plt.legend()
plt.show()
    